# Template de submissão

[![Launch Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/intrig-unicamp/hackathon5G/main?labpath=/challenges/submission-template.ipynb)
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/main/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/intrig-unicamp/hackathon5G/blob/main/challenges/submission-template.ipynb)

Esse Notebook importa todas as dependências necessárias para trabalhar com os conjuntos de dados fornecidos pela comissão organizadora da Hackathon SMARTNESS. As equipes poderão utilizá-lo como base para as submissões.

## Importando dependências globais

In [1]:
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics.pairwise import haversine_distances

import os
from datetime import datetime
from math import radians
import json
from urllib.parse import urlparse, parse_qsl
import random

random.seed(42)
pd.set_option('display.max_columns', None)

def count_decimal_places(num):
    str_num = str(num)
    if '.' in str_num:
        return len(str_num) - str_num.index('.') - 1
    else:
        return 0

pd.set_option('display.float_format', lambda x: '{:.{}f}'.format(x, min(count_decimal_places(x), 6)))

def pd_insert_beside(df, column_name, value):
    df.insert(df.columns.get_loc(value.name) + 1, column_name, value)

## Carregando os dados

In [2]:
DATA_PATH_PREFIX = '../datasets'
MOSAICO_PATH = os.path.join(DATA_PATH_PREFIX, 'mosaico')
GNETTRACKPRO_PATH = os.path.join(DATA_PATH_PREFIX, 'g-nettrack-pro')
YOUTUBE_QOE_PATH = os.path.join(DATA_PATH_PREFIX, 'youtube-qoe')

### Mosaico

In [3]:
datasets = [ os.path.join(MOSAICO_PATH, file) for file in os.listdir(MOSAICO_PATH) ]

ERBs = pd.concat([ pd.read_csv(dataset, encoding='iso-8859-1', low_memory=False) for dataset in datasets ], ignore_index=True).copy()
ERBs.replace({ 'Tecnologia': { 'NR ': 'NR', 'WDCMA': 'WCDMA' } }, inplace=True)
pd_insert_beside(ERBs, 'Tecnologia_gen', ERBs.Tecnologia.map({ 'GSM': '2G', 'WCDMA': '3G', 'LTE': '4G', 'NR': '5G' }))
ERBs.Azimute = ERBs.Azimute.str.replace(',', '.').replace('V', np.nan).astype(np.float64)
pd_insert_beside(ERBs, 'Azimute_rad', ERBs.Azimute.apply(lambda val: val * np.pi/180))
ERBs

,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,NumEstacao,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,Tecnologia,Tecnologia_gen,tipoTecnologia,meioAcesso,FreqTxMHz,FreqRxMHz,Azimute,Azimute_rad,CodTipoClasseEstacao,ClassInfraFisica,CompartilhamentoInfraFisica,CodTipoAntena,CodEquipamentoAntena,GanhoAntena,FrenteCostaAntena,AnguloMeiaPotenciaAntena,AnguloElevacao,Polarizacao,AlturaAntena,CodEquipamentoTransmissor,PotenciaTransmissorWatts,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado
0,LIC-LIC-01,CLARO S.A.,50409889580,10,16942008.0,2083922,Q 26,CONJUNTO 07,DF,5300108,5M00G7W,WCDMA,3G,NaN,NaN,2130.0,1940.0,NaN,NaN,FB,Greenfield,não,760,NaN,16.6,24.0,69,-1.0,X,40.0,001131003257,62.8,-15.91993,-47.9653,G,2023-02-14,2001-08-13,NaN,4d5c019f5deb1,2023-04-30,NaN,NaN
1,LIC-LIC-01,CLARO S.A.,50409889580,10,16942008.0,2083922,Q 26,CONJUNTO 07,DF,5300108,5M00G7W,WCDMA,3G,NaN,NaN,2130.0,1940.0,NaN,NaN,FB,Greenfield,não,760,NaN,16.6,24.0,69,-1.0,X,40.0,001131003257,62.8,-15.91993,-47.9653,G,2023-02-14,2001-08-13,NaN,4d5c019f5deb2,2023-04-30,NaN,NaN
2,LIC-LIC-01,CLARO S.A.,50409889580,10,16942008.0,2083922,Q 26,CONJUNTO 07,DF,5300108,5M00G7W,WCDMA,3G,NaN,NaN,2130.0,1940.0,NaN,NaN,FB,Greenfield,não,760,NaN,16.6,24.0,69,0.0,X,40.0,001131003257,62.8,-15.91993,-47.9653,G,2023-02-14,2001-08-13,NaN,4d5c019f5deb3,2023-04-30,NaN,NaN
3,LIC-LIC-01,CLARO S.A.,50409889580,10,16942008.0,2083922,Q 26,CONJUNTO 07,DF,5300108,5M00G7W,WCDMA,3G,NaN,NaN,2130.0,1940.0,NaN,NaN,FB,Greenfield,não,760,NaN,16.6,24.0,69,-1.0,X,40.0,001131003257,62.8,-15.91993,-47.9653,G,2023-02-14,2001-08-13,NaN,4d5c019f5deb4,2023-04-30,NaN,NaN
4,LIC-LIC-01,CLARO S.A.,50409889580,10,16942008.0,2083922,Q 26,CONJUNTO 07,DF,5300108,5M00G7W,WCDMA,3G,NaN,NaN,2130.0,1940.0,NaN,NaN,FB,Greenfield,não,760,NaN,16.6,24.0,69,-1.0,X,40.0,001131003257,62.8,-15.91993,-47.9653,G,2023-02-14,2001-08-13,NaN,4d5c019f5deb5,2023-04-30,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218865,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,LTE,4G,NaN,NaN,1862.5,1767.5,120.0,2.094395,FB,NaN,NaN,760,56001603518,15.0,25,56,2,X,38,040952002519,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c1,2029-12-08,NaN,NaN
1218866,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,LTE,4G,NaN,NaN,1862.5,1767.5,240.0,4.188790,FB,NaN,NaN,760,56001603518,15.0,25,56,2,X,38,040952002519,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c2,2029-12-08,NaN,NaN
1218867,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,41662008.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,WCDMA,3G,NaN,NaN,874.5,829.5,20.0,0.349066,FB,NaN,NaN,760,56001603518,14.5,25,65,2,X,38,000081201882,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c3,2029-12-08,NaN,NaN
1218868,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,41662008.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,WCDMA,3G,NaN,NaN,874.5,829.5,120.0,2.094395,FB,NaN,NaN,760,56001603518,14.5,25,65,2,X,38,000081201882,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c4,2029-12-08,NaN,NaN


### G-NetTrack

In [4]:
datasets = [ os.path.join(GNETTRACKPRO_PATH, file) for file in os.listdir(GNETTRACKPRO_PATH) ]

gnetDf = pd.concat([ pd.read_csv(dataset, sep='\t', low_memory=False, on_bad_lines='warn') for dataset in datasets ]).copy()
gnetDf.drop(gnetDf.loc[gnetDf.Timestamp == 'Timestamp'].index, inplace=True)
gnetDf.Timestamp = pd.to_datetime(gnetDf.Timestamp, format='%Y.%m.%d_%H.%M.%S')
pd_insert_beside(gnetDf, 'Timestamp_seconds', gnetDf.Timestamp.astype(int) / 10**9)
gnetDf.sort_values('Timestamp', inplace=True, ignore_index=True)
gnetDf.replace({
    'Altitude': { '--': np.nan },
    'CHARGING': { 'true': True, 'false': False },
    **{ k: { '--': np.nan } for k in ['Longitude', 'Latitude', 'NetworkTech', 'Accuracy'] },
    **{ k: { '-': np.nan } for k in ['CQI', 'SNR', 'Qual', 'LTERSSI'] },
}, inplace=True)
gnetDf = gnetDf.astype({
    **{ k: 'string' for k in ['NetworkTech', 'NetworkMode', 'Location', 'State', 'EVENT', 'CGI', 'Node', 'CellID', 'LAC', 'Operatorname', 'Operator', 'IP'] },
    **{ k: np.float64 for k in ['Longitude', 'Latitude', 'Speed', 'Accuracy', 'SNR', 'Level', 'Qual', 'CQI', 'LTERSSI', 'ARFCN', 'DL_bitrate', 'UL_bitrate', 'PSC', 'Altitude', 'Height'] },
    **{ k: np.int32 for k in ['BATTERY'] },
})
gnetDf.replace({
    'Altitude': { 0: np.nan },
    'Height': { 0: np.nan, -10000: np.nan },
    'Speed': { -99: np.nan },
}, inplace=True)
gnetDf

Skipping line 1291: expected 260 fields, saw 390
Skipping line 1292: expected 260 fields, saw 390
Skipping line 1293: expected 260 fields, saw 390
Skipping line 1294: expected 260 fields, saw 390
Skipping line 1295: expected 260 fields, saw 390
Skipping line 1296: expected 260 fields, saw 390
Skipping line 1297: expected 260 fields, saw 390
Skipping line 1298: expected 260 fields, saw 390
Skipping line 1299: expected 260 fields, saw 390
Skipping line 1300: expected 260 fields, saw 390
Skipping line 1301: expected 260 fields, saw 390
Skipping line 1302: expected 260 fields, saw 390
Skipping line 1303: expected 260 fields, saw 390
Skipping line 1304: expected 260 fields, saw 390
Skipping line 1305: expected 260 fields, saw 390
Skipping line 1306: expected 260 fields, saw 390
Skipping line 1307: expected 260 fields, saw 390
Skipping line 1308: expected 260 fields, saw 390
Skipping line 1309: expected 260 fields, saw 390
Skipping line 1310: expected 260 fields, saw 390
Skipping line 1311: 

,Timestamp,Timestamp_seconds,Longitude,Latitude,Speed,Operatorname,Operator,CGI,Cellname,Node,CellID,LAC,NetworkTech,NetworkMode,Level,Qual,SNR,CQI,LTERSSI,ARFCN,DL_bitrate,UL_bitrate,PSC,Altitude,Height,Accuracy,Location,State,PINGAVG,PINGMIN,PINGMAX,PINGSTDEV,PINGLOSS,TESTDOWNLINK,TESTUPLINK,TESTDOWNLINKMAX,TESTUPLINKMAX,Test_Status,DataConnection_Type,DataConnection_Info,Layer,Filemark,IMEI,Distance,Bearing,IP,TA,EVENT,EVENTDETAILS,IMSI,MSISDN,SERVINGTIME,CELLHEX,NODEHEX,LACHEX,RAWCELLID,VERSION_NAME,VERSION_CODE,DEVICE,BAND,BANDWIDTH,INFO,SecondCell_NODE,SecondCell_CELLID,SecondCell_RSRP,SecondCell_RSRQ,SecondCell_SNR,CSI_RSRP,CSI_RSRQ,CSI_SNR,NetworkTypeNum,CA,SecondCell_PSC,SecondCell_ARFCN,SecondCell_BAND,Site_BEARING,BATTERY,CHARGING,NRSTATE,R4,NTech1,NCellName1,NCellid1,NLAC1,NCell1,NARFCN1,NRxLev1,NQual1,NDistance1,NBearing1,NTech2,NCellName2,NCellid2,NLAC2,NCell2,NARFCN2,NRxLev2,NQual2,NDistance2,NBearing2,NTech3,NCellName3,NCellid3,NLAC3,NCell3,NARFCN3,NRxLev3,NQual3,NDistance3,NBearing3,NTech4,NCellName4,NCellid4,NLAC4,NCell4,NARFCN4,NRxLev4,NQual4,NDistance4,NBearing4,NTech5,NCellName5,NCellid5,NLAC5,NCell5,NARFCN5,NRxLev5,NQual5,NDistance5,NBearing5,NTech6,NCellName6,NCellid6,NLAC6,NCell6,NARFCN6,NRxLev6,NQual6,NDistance6,NBearing6,NTech7,NCellName7,NCellid7,NLAC7,NCell7,NARFCN7,NRxLev7,NQual7,NDistance7,NBearing7,NTech8,NCellName8,NCellid8,NLAC8,NCell8,NARFCN8,NRxLev8,NQual8,NDistance8,NBearing8,NTech9,NCellName9,NCellid9,NLAC9,NCell9,NARFCN9,NRxLev9,NQual9,NDistance9,NBearing9,NTech10,NCellName10,NCellid10,NLAC10,NCell10,NARFCN10,NRxLev10,NQual10,NDistance10,NBearing10,NTech11,NCellName11,NCellid11,NLAC11,NCell11,NARFCN11,NRxLev11,NQual11,NDistance11,NBearing11,NTech12,NCellName12,NCellid12,NLAC12,NCell12,NARFCN12,NRxLev12,NQual12,NDistance12,NBearing12,NTech13,NCellName13,NCellid13,NLAC13,NCell13,NARFCN13,NRxLev13,NQual13,NDistance13,NBearing13,NTech14,NCellName14,NCellid14,NLAC14,NCell14,NARFCN14,NRxLev14,NQual14,NDistance14,NBearing14,NTech15,NCellName15,NCellid15,NLAC15,NCell15,NARFCN15,NRxLev15,NQual15,NDistance15,NBearing15,NTech16,NCellName16,NCellid16,NLAC16,NCell16,NARFCN16,NRxLev16,NQual16,NDistance16,NBearing16,NTech17,NCellName17,NCellid17,NLAC17,NCell17,NARFCN17,NRxLev17,NQual17,NDistance17,NBearing17,NTech18,NCellName18,NCellid18,NLAC18,NCell18,NARFCN18,NRxLev18,NQual18,NDistance18,NBearing18,Unnamed: 259
0,2023-01-21 14:44:46,1674312286.0,NaN,NaN,NaN,CLARO_BR,72405,724056481100018,NaN,113726,18,64811,4G,LTE,-110.0,-8.0,16.0,14.0,-61.0,1400.0,25.0,29.0,111.0,NaN,NaN,NaN,<NA>,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,13,1,NaN,NaN,0,0,10.130.154.201,31.0,LOG START,NaN,NaN,NaN,8,12,1BC3E,FD2B,29113874,27.5,535,samsung:SM-G991B:TP1A.220624.014,L3,10/20/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,False,NaN,NaN,NaN,0,81,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-21 14:44:46,1674312286.0,NaN,NaN,NaN,CLARO_BR,72405,724056481100018,NaN,113726,18,64811,4G,LTE,-112.0,-8.0,17.0,14.0,-65.0,1400.0,119.0,53.0,111.0,NaN,NaN,NaN,<NA>,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,13,1,NaN,NaN,0,0,10.130.154.201,31.0,PERIODIC,NaN,NaN,NaN,9,12,1BC3E,FD2B,29113874,27.5,535,samsung:SM-G991B:TP1A.220624.014,L3,10/20/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,False,NaN,NaN,NaN,0,81,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### YouTube QoE

In [5]:
datasets = [ os.path.join(YOUTUBE_QOE_PATH, file) for file in os.listdir(YOUTUBE_QOE_PATH) ]

ytDf = pd.concat([ pd.read_pickle(dataset) for dataset in datasets ], ignore_index=True).copy()

ytDf.sort_values('time', inplace=True, ignore_index=True)